In [ ]:
import os
import random
import shutil

def split_yolo_dataset(
    labels_dir="data/labels",
    images_dir="NepaliDevanagariText",
    output_dir="data",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
):
    """
    Splits annotated YOLO OBB dataset into train/val/test sets in 8:1:1 ratio.
    - labels_dir: where YOLO .txt annotation files live
    - images_dir: where the actual images are stored
    - output_dir: where the new YOLO dataset will be created
    """


    # Create output structure
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

    # Get all annotation files
    label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]
    random.shuffle(label_files)

    # Split sizes
    total = len(label_files)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)

    splits = {
        "train": label_files[:train_end],
        "val": label_files[train_end:val_end],
        "test": label_files[val_end:]
    }

    # Copy files
    for split, files in splits.items():
        for label_file in files:
            label_src = os.path.join(labels_dir, label_file)
            label_dst = os.path.join(output_dir, "labels", split, label_file)

            shutil.copy(label_src, label_dst)

            # Find corresponding image
            base_name = os.path.splitext(label_file)[0]
            img_extensions = [".jpg", ".jpeg", ".png"]

            img_found = False
            for ext in img_extensions:
                img_src = os.path.join(images_dir, base_name + ext)
                if os.path.exists(img_src):
                    img_dst = os.path.join(output_dir, "images", split, base_name + ext)
                    shutil.copy(img_src, img_dst)
                    img_found = True
                    break

            if not img_found:
                print(f"[WARNING] No image found for {label_file}")

    print(f"✅ Dataset split complete! Saved in '{output_dir}'")
    print(f"Train: {len(splits['train'])}, Val: {len(splits['val'])}, Test: {len(splits['test'])}")

split_yolo_dataset()


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/labels'

In [1]:
import os
import shutil
%conda install -n .conda ipykernel
%pip install pyyaml


Note: you may need to restart the kernel to use updated packages.



EnvironmentLocationNotFound: Not a conda environment: C:\Users\Nikunj\miniforge3\envs\.conda



Note: you may need to restart the kernel to use updated packages.


In [2]:
import yaml


In [4]:
# def create_yolo_yaml_config(yaml_filepath, dataset_path, dataset_labels):

#     data = {'path':dataset_path,
#             'train': os.path.join('images', 'train'),
#             'validation': os.path.join('images', 'validation'),
#             'names':{i:label for i, label in enumerate(dataset_labels)}
#             }

#     # Save the changes to the file
#     with open(yaml_filepath, 'w') as fp:
#         yaml.dump(data, fp, sort_keys=False)


def create_yolo_yaml_config(
    yaml_filepath,
    dataset_path,
    dataset_labels,
    train_split="images/train",
    val_split="images/val",
    test_split="images/test",
    nc=None,
    task="detect",        # "detect", "segment", "classify", "obb"
    model="yolov8n.pt",   # default model, can change to yolov8s.pt, yolov8m.pt, etc.
    epochs=100,
    batch=16,
    img_size=960,         # changed from 640 to 960
    lr0=0.01,
    optimizer="SGD"       # SGD, Adam, AdamW, RMSProp
):
    """
    Create a YOLOv8 YAML dataset config with customizable training parameters.
    Ensures letterbox padding will be used during training/inference to preserve aspect ratio.
    """

    if nc is None:
        nc = len(dataset_labels)

    data = {
        # dataset setup
        "path": dataset_path,
        "train": train_split,
        "val": val_split,
        "test": test_split,
        "nc": nc,
        "names": {i: label for i, label in enumerate(dataset_labels)},

        # training settings
        "task": task,
        "model": model,
        "epochs": epochs,
        "batch": batch,
        "imgsz": img_size,    # YOLO will resize the longer side to 960 and pad the shorter side
        "lr0": lr0,
        "optimizer": optimizer,

        # augmentation defaults
        "hsv_h": 0.015,
        "hsv_s": 0.7,
        "hsv_v": 0.4,
        "flipud": 0.0,
        "fliplr": 0.5,
        #"mosaic": 1.0,
        #"mixup": 0.1,

        # ensure letterbox scaling (not in YAML directly, handled by trainer)
        "rect": True,   # True would train with rectangular inputs
        "pad": True      # pad to square 960x960 without stretching
    }

    with open(yaml_filepath, "w") as fp:
        yaml.dump(data, fp, sort_keys=False)

    print(f"✅ YOLO config YAML saved to {yaml_filepath} with 960x960 letterbox scaling")


In [5]:
names = ['line']
dataset_path = os.path.abspath(os.path.join('.', 'data')) # recommended to use absolute path 
yaml_filepath = os.path.join('.', 'config.yaml')
create_yolo_yaml_config(yaml_filepath, dataset_path, names)
class_ids = 0

✅ YOLO config YAML saved to .\config.yaml with 960x960 letterbox scaling


In [6]:
print(dataset_path, yaml_filepath)

c:\Users\Nikunj\Desktop\Code\NepaliDevanagariVision\Object_Detection\data .\config.yaml


In [7]:
#%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.202  Python-3.12.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Setup complete  (16 CPUs, 15.9 GB RAM, 999.5/1862.2 GB disk)


In [8]:
from ultralytics import YOLO


model = YOLO("yolov8m.pt")

results = model.train(data='./config.yaml', 
                      epochs=100)


New https://pypi.org/project/ultralytics/8.3.229 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.202  Python-3.12.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None,